In [1]:
import sys 

sys.path.insert(1, "C:/Users/adamf/Cropharvest/crop-mask/src")

import map_comparison as mp

In [2]:
mp.run_comparison('C:/Users/adamf/Cropharvest/cropvalidation/data/HarvestData/togo.csv', 'C:/Users/adamf/Cropharvest/data/TogoBinaryMaps/ASAP_Togo_Binary.tif')

C:\Users\adamf\anaconda3\envs\landcover-mapping\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


TypeError: 'generator' object is not subscriptable

In [19]:
from typing import List
import geopandas as gpd
import numpy as np
import sklearn.metrics
import rasterio as ras
from pyproj import Proj, transform
import pyproj

import fiona
fiona.supported_drivers
fiona.drvsupport.supported_drivers['csv'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['CSV'] = 'rw' # enable KML support which is disabled by default

cropmap = 'C:/Users/adamf/Cropharvest/data/TogoBinaryMaps/ASAP_Togo_Binary.tif'
validation = 'C:/Users/adamf/Cropharvest/cropvalidation/data/HarvestData/togo.csv'

report = []
cropmap = ras.open(cropmap)
validation = gpd.read_file(validation)

validation.crs = Proj("+init=epsg:4326")

# validation = validation[validation['subset'] == 'testing']
# validation.loc[validation['crop_probability'].astype(float) >= 0.5, 'crop_probability'] = 1
# validation.loc[validation['crop_probability'].astype(float) < 0.5, 'crop_probability'] = 0

# def reproject(latIn, lonIn, ProjIn, ProjOut) -> np.array:
#     if latIn.size != latIn.size:
#         raise Exception("Input arrays are not of equal size.")

#     latOut = []
#     lonOut = []

#     for coordinate in np.arange(latIn.size):
#         x1,y1 = latIn[coordinate],lonIn[coordinate]
#         newLat, newLon = transform(ProjIn,ProjOut,x1,y1)
#         latOut.append(newLat), lonOut.append(newLon)

#     return np.array(latOut), np.array(lonOut)

# newLat, newLon = reproject(validation['lat'].values, validation['lon'].values, Proj('+init=epsg:4326'), Proj(cropmap.crs))
# validation['lat'] = newLat
# validation['lon'] = newLon

cropmap_sampled = ras.sample.sample_gen(cropmap, ((float(x),float(y)) for x,y in zip(validation['lat'], validation['lon'])))
cropmap_sampled = gpd.GeoSeries([x.astype(float) for x in cropmap_sampled])


target_names = ['non_crop', 'crop']
class_report = sklearn.metrics.classification_report(validation['crop_probability'], cropmap_sampled, target_names = target_names, output_dict=True)
accuracy = sklearn.metrics.accuracy_score(validation['crop_probability'], cropmap_sampled)

report = [accuracy, class_report['crop']['precision'], class_report['non_crop']['precision'], class_report['non_crop']['recall'], class_report['crop']['recall']]




C:\Users\adamf\anaconda3\envs\landcover-mapping\lib\site-packages\ipykernel_launcher.py:46: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.


ValueError: Classification metrics can't handle a mix of multiclass and unknown targets

In [24]:
validation = validation.to_crs({'init': 'epsg:4326'})

AttributeError: 'Proj' object has no attribute 'items'

In [9]:
validation['lat']

77      1.363956e+06
116     1.368445e+06
138     1.350216e+06
140     1.397357e+06
145     1.345965e+06
            ...     
1537    9.797879e+05
1544    8.727081e+05
1561    1.118807e+06
1605    8.711022e+05
1630    8.672357e+05
Name: lat, Length: 355, dtype: float64